In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

plt.rcParams['font.family'] = 'HYGraphic-Medium'

In [2]:
df = {}

for i in range(2010, 2022):
    df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
    df[i].drop(labels=df[i][df[i]['아내지역코드'] > 39].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편지역코드'] > 39].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내직업코드'] == 99].index, inplace=True)

C:\Users\nexen\AppData\Local\Temp\ipykernel_19496\3438226185.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\nexen\AppData\Local\Temp\ipykernel_19496\3438226185.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\nexen\AppData\Local\Temp\ipykernel_19496\3438226185.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
C:\Users\nexen\AppData\Local\Temp\ipykernel_19496\3438226185.py:4: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")


In [76]:
for year in range(2010, 2022):
    df[year].loc[df[year]['남편연령'] > df[year]['아내연령'], "결혼 연령차 유형"] = "남성 연상혼" 
    df[year].loc[df[year]['남편연령'] < df[year]['아내연령'], "결혼 연령차 유형"] = "여성 연상혼" 
    df[year].loc[df[year]['남편연령'].astype(int) == df[year]['아내연령'].astype(int), "결혼 연령차 유형"] = "동갑혼" 

    df[year].loc[df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), "아내 기준 대도시 유무"] = "대도시 거주" 
    df[year].loc[~df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), "아내 기준 대도시 유무"] = "대도시 비거주" 

    df[year].loc[df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]), "남편 기준 대도시 유무"] = "대도시 거주" 
    df[year].loc[~df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]), "남편 기준 대도시 유무"] = "대도시 비거주" 
    
    df[year].loc[df[year]["결혼 연령차 유형"] == "여성 연상혼" , "연상혼 여부"] = 1
    df[year].loc[~(df[year]["결혼 연령차 유형"] == "여성 연상혼"), "연상혼 여부"] = 0

    df[year].loc[df[year]['아내교육코드'] > df[year]['남편교육코드'], "강혼 여부"] = 1
    df[year].loc[~(df[year]['아내교육코드'] > df[year]['남편교육코드']), "강혼 여부"] = 0

    #여성 대도시 거주 유무와 결혼 연령차 유형 연관성 높음
    #남편 대도시 거주 유무와 결혼 연령차 유형 2021 제외 연관성 높음

    years = [2018, 2019, 2020]

### 연상혼 / 소득 관계

In [78]:
for y in years:
    crosstab = pd.crosstab(df[y]['아내직업코드'], df[y]["연상혼 여부"])
    c, p, dof, expected = scipy.stats.chi2_contingency(crosstab)

    print(f"-------------- {y} 연도 연상혼 여부 - 여성 소득 카이제곱 검정 -----------------")
    print("c-val : " + str(c))
    print("P-val : " + str(p))
    print("dof : " + str(dof))


-------------- 2018 연도 연상혼 여부 - 여성 소득 카이제곱 검정 -----------------
c-val : 449.72506454015104
P-val : 4.2349666162336154e-92
dof : 8
-------------- 2019 연도 연상혼 여부 - 여성 소득 카이제곱 검정 -----------------
c-val : 287.5818386246488
P-val : 1.8052292020653766e-57
dof : 8
-------------- 2020 연도 연상혼 여부 - 여성 소득 카이제곱 검정 -----------------
c-val : 223.3776808645135
P-val : 7.442992539024987e-44
dof : 8


In [ ]:

for y in range(2010, 2022):
    plt.figure()
    crosstab = pd.crosstab(df[y]['아내직업코드'], df[y]["연상혼 여부"], normalize='columns')
    crosstab.plot.bar()
    plt.title(str(y))

# 강혼과 고소득은 양의 상관관계라고 보기 힘듦 


### 대도시 거주 / 소득 관계

In [ ]:

# 대체적으로 고소득 (1, 2, 3) -> 대도시 거주 여성 중 퍼센테이지 높음
# 각각 대도시 거주 중 / 대도시 비거주 중 해당 직업 퍼센테이지

for y in range(2010, 2022):
    plt.figure()
    crosstab = pd.crosstab(df[y]['아내직업코드'], df[y]["아내 기준 대도시 유무"], normalize='columns')
    crosstab.plot.bar()
    plt.title(str(y))


In [71]:
for y in years:
    crosstab = pd.crosstab(df[y]['아내직업코드'], df[y]["아내 기준 대도시 유무"])
    c, p, dof, expected = scipy.stats.chi2_contingency(crosstab)

    print(f"-------------- {y} 연도 아내 대도시 거주 여부 - 소득 카이제곱 검정 -----------------")
    print("c-val : " + str(c))
    print("P-val : " + str(p))
    print("dof : " + str(dof))

-------------- 2018 연도 아내 대도시 거주 여부 - 소득 카이제곱 검정 -----------------
c-val : 1788.6536081792397
P-val : 0.0
dof : 8
-------------- 2019 연도 아내 대도시 거주 여부 - 소득 카이제곱 검정 -----------------
c-val : 1566.198287470004
P-val : 0.0
dof : 8
-------------- 2020 연도 아내 대도시 거주 여부 - 소득 카이제곱 검정 -----------------
c-val : 1441.0608715358505
P-val : 7.485536341892138e-306
dof : 8


In [ ]:
# 직업 코드 1 -> 대도시 비거주 우세 / 2, 3 대도시 거주 우세
# 각각 대도시 거주 중 / 대도시 비거주 중 해당 직업 퍼센테이지

for y in range(2010, 2022):
    plt.figure()
    crosstab = pd.crosstab(df[y]['남편직업코드'], df[y]["남편 기준 대도시 유무"], normalize='columns')
    crosstab.plot.bar()
    plt.title(str(y))



In [75]:
for y in years:
    crosstab = pd.crosstab(df[y]['남편직업코드'], df[y]["남편 기준 대도시 유무"])
    c, p, dof, expected = scipy.stats.chi2_contingency(crosstab)

    print(f"-------------- {y} 연도 남편 대도시 거주 여부 - 남편 소득 카이제곱 검정 -----------------")
    print("c-val : " + str(c))
    print("P-val : " + str(p))
    print("dof : " + str(dof))

-------------- 2018 연도 남편 대도시 거주 여부 - 남편 소득 카이제곱 검정 -----------------
c-val : 3825.9800477523645
P-val : 0.0
dof : 9
-------------- 2019 연도 남편 대도시 거주 여부 - 남편 소득 카이제곱 검정 -----------------
c-val : 3603.6508923051283
P-val : 0.0
dof : 9
-------------- 2020 연도 남편 대도시 거주 여부 - 남편 소득 카이제곱 검정 -----------------
c-val : 3139.9417216175807
P-val : 0.0
dof : 9


### 연상혼 - 소득 관계

In [ ]:
for y in years:
    crosstab = pd.crosstab(df[y]['남편직업코드'], df[y]["남편 기준 대도시 유무"])
    c, p, dof, expected = scipy.stats.chi2_contingency(crosstab)

    print(f"-------------- {y} 연도 남편 대도시 거주 여부 - 남편 소득 카이제곱 검정 -----------------")
    print("c-val : " + str(c))
    print("P-val : " + str(p))
    print("dof : " + str(dof))